In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import string

## Lets load the dataset

In [32]:
df = pd.read_csv('C:/Users/hp/Desktop/Test/incidents_reseau.csv')
df

,ID d'incident,Timestamp,Type_incident,Description,Severité,Statut,Responsable
0,1,2023-10-13,Mise à jour échouée,Échec de l'application d'une mise à jour.,Critique,Résolu,Equipe Systèmes
1,2,2023-09-15,Problème de connexion,Connexion instable ou interrompue.,Critique,Ouvert,Equipe Systèmes
2,3,2023-11-14,Problème de connexion,Connexion instable ou interrompue.,Critique,En cours,Equipe Support
3,4,2023-10-12,Intrusion,Tentative d'accès non autorisé détectée.,Critique,En cours,Equipe Réseau
4,5,2023-08-06,Surcharge,Capacité du réseau dépassée.,Faible,Résolu,Equipe Réseau
...,...,...,...,...,...,...,...
2995,2996,2023-04-21,Intrusion,Tentative d'accès non autorisé détectée.,Moyenne,Résolu,Equipe Systèmes
2996,2997,2023-04-14,Problème de connexion,Connexion instable ou interrompue.,Élevée,En cours,Equipe Sécurité
2997,2998,2023-07-03,Problème de connexion,Connexion instable ou interrompue.,Critique,Ouvert,Equipe Sécurité
2998,2999,2023-07-22,Surcharge,Capacité du réseau dépassée.,Moyenne,Ouvert,Equipe Réseau


## Data Preprocessing

In [33]:
# Convertir les timestamps en datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [34]:
# Traitement de texte
# Télécharger les stopwords de NLTK
nltk.download('stopwords')
stop_words = set(stopwords.words('french'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
def preprocess_text(text):
    # Convertir en minuscules
    text = text.lower()
    # Supprimer la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Supprimer les stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

df['Description'] = df['Description'].apply(preprocess_text)

In [36]:
df

,ID d'incident,Timestamp,Type_incident,Description,Severité,Statut,Responsable
0,1,2023-10-13,Mise à jour échouée,échec lapplication dune mise jour,Critique,Résolu,Equipe Systèmes
1,2,2023-09-15,Problème de connexion,connexion instable interrompue,Critique,Ouvert,Equipe Systèmes
2,3,2023-11-14,Problème de connexion,connexion instable interrompue,Critique,En cours,Equipe Support
3,4,2023-10-12,Intrusion,tentative daccès non autorisé détectée,Critique,En cours,Equipe Réseau
4,5,2023-08-06,Surcharge,capacité réseau dépassée,Faible,Résolu,Equipe Réseau
...,...,...,...,...,...,...,...
2995,2996,2023-04-21,Intrusion,tentative daccès non autorisé détectée,Moyenne,Résolu,Equipe Systèmes
2996,2997,2023-04-14,Problème de connexion,connexion instable interrompue,Élevée,En cours,Equipe Sécurité
2997,2998,2023-07-03,Problème de connexion,connexion instable interrompue,Critique,Ouvert,Equipe Sécurité
2998,2999,2023-07-22,Surcharge,capacité réseau dépassée,Moyenne,Ouvert,Equipe Réseau


In [37]:
# Vectorisation des descriptions
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Description'])

## EDA


## Train Test Split


In [38]:
# Target
y = df['Type_incident']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


## Model Training

In [40]:
models = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=200),
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(n_estimators=100)
}

In [41]:
# Entrainer et evaluer chaque modele
results = {}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results[model_name] = {
        "accuracy": accuracy,
        "report": report
    }

In [42]:
# Results
for model_name, metrics in results.items():
    print(f"Model: {model_name}")
    print(f"Accuracy: {metrics['accuracy']}")
    print("Classification Report:")
    print(metrics['report'])
    print("="*60)

Model: Naive Bayes
Accuracy: 1.0
Classification Report:
                       precision    recall  f1-score   support

            Intrusion       1.00      1.00      1.00       175
  Mise à jour échouée       1.00      1.00      1.00       191
     Panne matérielle       1.00      1.00      1.00       164
Problème de connexion       1.00      1.00      1.00       185
            Surcharge       1.00      1.00      1.00       185

             accuracy                           1.00       900
            macro avg       1.00      1.00      1.00       900
         weighted avg       1.00      1.00      1.00       900

Model: Logistic Regression
Accuracy: 1.0
Classification Report:
                       precision    recall  f1-score   support

            Intrusion       1.00      1.00      1.00       175
  Mise à jour échouée       1.00      1.00      1.00       191
     Panne matérielle       1.00      1.00      1.00       164
Problème de connexion       1.00      1.00      1.00     

## Pickling The Model file For Deployment

In [43]:
import pickle

In [47]:
for model_name, model in models.items():

    with open(f'C:/Users/hp/Desktop/Test/{model_name}_model.pkl', 'wb') as file:
        pickle.dump(model, file)

# Sauvegarder le vectorizer
with open(f'C:/Users/hp/Desktop/Test/vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

print("Models and vectorizer have been saved.")

Models and vectorizer have been saved.
